In [2]:
import json
import pandas as pd
from tqdm import tqdm

## 处理数据集

处理交互信息

In [3]:
file = "D:\work_space\Office_Products.jsonl"
with open(file, 'r') as fp:
    for line in tqdm(fp):
        data = json.loads(line.strip())
        user_id = data['user_id']
        asin = data['asin'] 
        rating = data['rating']
        timestamp = data['timestamp']

        with open('D:/Code/graduation_design/data/Office_Products/interaction.dat', 'a') as f:
            f.write(f"{user_id}::{asin}::{rating}::{timestamp}\n")

0it [00:00, ?it/s]

12845712it [51:03, 4193.05it/s]


处理物品信息

In [37]:
file = "D:\work_space\meta_Office_Products.jsonl"
with open(file, 'r') as fp:
    for line in tqdm(fp):
        data = json.loads(line.strip())
        parent_asin = data['parent_asin']
        with open('D:/Code/graduation_design/data/Office_Products/item.dat', 'a') as f:
            f.write(f"{parent_asin}:::{json.dumps(data)}\n")

683223it [03:41, 2854.52it/s]

## 对用户物品交互数据集进行筛选

读取数据集

In [4]:
df = pd.read_csv("D:\Code\graduation_design\data\kuairec\interaction.csv", sep=",", engine='python')
df.columns = ['user', 'item', 'watch_ratio', 'timestamp']
df

,user,item,watah_ratio,timestamp
0,14,148,0.722103,1.593898e+09
1,14,183,1.907377,1.593898e+09
2,14,3649,2.063311,1.593898e+09
3,14,5262,0.566388,1.593898e+09
4,14,8234,0.418364,1.593899e+09
...,...,...,...,...
4676565,7162,2267,2.178160,NaN
4676566,7162,2065,1.964562,NaN
4676567,7162,1296,0.839960,NaN
4676568,7162,4822,0.486148,NaN


In [5]:
df = df[df['timestamp'].notna()]
df

,user,item,watah_ratio,timestamp
0,14,148,0.722103,1.593898e+09
1,14,183,1.907377,1.593898e+09
2,14,3649,2.063311,1.593898e+09
3,14,5262,0.566388,1.593898e+09
4,14,8234,0.418364,1.593899e+09
...,...,...,...,...
4676370,7162,9177,0.142857,1.598962e+09
4676371,7162,4987,1.234848,1.599029e+09
4676372,7162,7988,1.024412,1.599094e+09
4676373,7162,6533,0.273750,1.599231e+09


In [10]:
ratio_count = df[df['watah_ratio'] >= 0.7].shape[0]
total_count = df.shape[0]
ratio_percentage = (ratio_count / total_count) * 100 if total_count > 0 else 0
print(f"watah_ratio大于等于1的比例: {ratio_percentage:.2f}%")

watah_ratio大于等于1的比例: 56.44%


In [14]:
filter_df = df[df['watah_ratio'] >= 0.7]
filter_df

,user,item,watah_ratio,timestamp
0,14,148,0.722103,1.593898e+09
1,14,183,1.907377,1.593898e+09
2,14,3649,2.063311,1.593898e+09
6,14,1963,0.898123,1.593899e+09
9,14,171,1.632739,1.593900e+09
...,...,...,...,...
4676366,7162,4978,1.527117,1.598956e+09
4676368,7162,3360,1.457282,1.598960e+09
4676371,7162,4987,1.234848,1.599029e+09
4676372,7162,7988,1.024412,1.599094e+09


In [18]:
average_interactions = filter_df.groupby('user').size().mean()
print(f"平均每个用户的交互数量: {average_interactions:.2f}")
average_interactions = filter_df.groupby('item').size().mean()
print(f"平均每个物品的交互数量: {average_interactions:.2f}")

平均每个用户的交互数量: 1797.93
平均每个物品的交互数量: 762.97


In [19]:
filter_df.to_csv("D:/Code/graduation_design/data/kuairec/interaction.dat", index=False)

In [16]:
ratio_df = filter_df
item_counts = ratio_df['item'].value_counts()
items_to_keep = item_counts[item_counts >= 20].index
ratio_df = ratio_df[ratio_df['item'].isin(items_to_keep)]

user_counts = ratio_df['user'].value_counts()
users_to_keep = user_counts[user_counts >= 30].index
ratio_df = ratio_df[ratio_df['user'].isin(users_to_keep)]

print(f"用户数量: {len(ratio_df['user'].unique())}, 物品数量: {len(ratio_df['item'].unique())}")

ratio_df = ratio_df.sort_values(['user', 'timestamp'])
ratio_df = ratio_df.reset_index(drop=True)
ratio_df

用户数量: 1411, 物品数量: 3146


,user,item,watah_ratio,timestamp
0,14,148,0.722103,1.593898e+09
1,14,183,1.907377,1.593898e+09
2,14,3649,2.063311,1.593898e+09
3,14,1963,0.898123,1.593899e+09
4,14,171,1.632739,1.593900e+09
...,...,...,...,...
2535185,7162,1331,0.966409,1.598952e+09
2535186,7162,4978,1.527117,1.598956e+09
2535187,7162,3360,1.457282,1.598960e+09
2535188,7162,4987,1.234848,1.599029e+09


In [17]:
average_interactions = ratio_df.groupby('user').size().mean()
print(f"平均每个用户的交互数量: {average_interactions:.2f}")

平均每个用户的交互数量: 1796.73


In [ ]:
data.to_csv("D:/Code/graduation_design/data/Office_Products/processed_interaction.dat", index=False)

## 处理item.dat数据集

### 用于基础模型训练

In [21]:
df = pd.read_csv("D:\Code\graduation_design\data\kuairec\data\kuairec_caption_category.csv", sep=",", engine='python')
df

,video_id,manual_cover_text,caption,topic_tag,first_level_category_id,first_level_category_name,second_level_category_id,second_level_category_name,third_level_category_id,third_level_category_name
0,0,UNKNOWN,精神小伙路难走 程哥你狗粮慢点撒,[],8.0,颜值,673.0,颜值随拍,-124.0,UNKNOWN
1,1,UNKNOWN,NaN,[],27.0,高新数码,-124.0,UNKNOWN,-124.0,UNKNOWN
2,2,UNKNOWN,晚饭后，运动一下！,[],9.0,喜剧,727.0,搞笑互动,-124.0,UNKNOWN
3,3,UNKNOWN,我平淡无奇，惊艳不了时光，温柔不了岁月，我只想漫无目的的走走，努力发笔小财，给自己买花 自己长大.,[],26.0,摄影,686.0,主题摄影,2434.0,景物摄影
4,4,五爱街最美美女 一天1q,#搞笑 #感谢快手我要上热门 #五爱市场 这真是完美搭配啊！,"[五爱市场,感谢快手我要上热门,搞笑]",5.0,时尚,737.0,营销售卖,2596.0,女装
...,...,...,...,...,...,...,...,...,...,...
10727,10723,UNKNOWN,昨天爱你，今天爱你，明天也爱你，丫头，别担心，我以后都会爱你，我的小傻瓜@公主没烦恼 、(O...,[],33.0,自拍,-124.0,UNKNOWN,-124.0,UNKNOWN
10728,10724,UNKNOWN,#感谢推广小助手 #感谢快手绿色平台 #,"[感谢快手绿色平台,感谢推广小助手]",6.0,明星娱乐,-124.0,UNKNOWN,-124.0,UNKNOWN
10729,10725,UNKNOWN,NaN,[],15.0,艺术,170.0,表演,-124.0,UNKNOWN
10730,10726,老人言,老人言，喜欢留个关注加红心 #老人言 @今天拍点啥(O840386039) @快手活动中...,[老人言],38.0,读书,696.0,文学赏析,2477.0,民间俗语


In [22]:
selected_columns = df[['video_id', 'caption']]
selected_columns.to_csv('output.csv', index=False)